bs4, uc drive, selenium, deep-translator, snow-nlp, schedule, spacytextblob, spacy
options.add_argument('--disable-blink-features=AutomationControlled')


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_experimental_option(
    "excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_argument('lang=zh-CN,zh,zh-TW,en-US,en')
chrome_options.add_argument(
    'user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36')
chrome_options.add_argument("disable-blink-features=AutomationControlled")
chrome_options.add_argument("proxy-server=socks5://127.0.0.1:1080")
preferences = {
    "webrtc.ip_handling_policy": "disable_non_proxied_udp",
    "webrtc.multiple_routes_enabled": False,
    "webrtc.nonproxied_udp_enabled": False
}

driver = webdriver.Chrome(chrome_options=chrome_options)

In [1]:
from bs4 import BeautifulSoup
import langid
import pandas as pd
import requests
import sys
import time
import requests
from itertools import cycle
from datetime import datetime
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchWindowException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import undetected_chromedriver as uc
from datetime import timedelta, date, datetime
from dateutil.relativedelta import relativedelta
from snownlp import SnowNLP
import pycantonese
from pycantonese.word_segmentation import Segmenter
from deep_translator import GoogleTranslator
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import nltk
nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/wslhui/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
#functions
def click_thread(driver, idx):
    #click thread
    auto_xpath = f'//*[@id="leftPanel"]/div[2]/div[{idx + 1}]/div/div[2]/a[1]'
    try:
        driver.find_element(By.XPATH, auto_xpath).click()
        #print(idx)
    except:
        #print("skip")
        pass
    time.sleep(0.3)
    
def scroll_left_panel(driver):
    counter = 0
    while True:
        html = driver.page_source
        soup = BeautifulSoup(html)
        checker = soup.find_all("div",{"class":"_21IQKhlBjN2jlHS_TVgI3l"})
        
        counter = counter + 1 
        previous_checker = len(checker)
        if counter == 10:
            try:
                if len(checker) == previous_checker:
                    break
            except:
                pass

        try:
            driver.find_element(By.CLASS_NAME, '_33r1FGqGJZF-fM1VZm7mhN').text
            time.sleep(2)
            break
        except:
            pass

        for i in range(20):
            ActionChains(driver)\
                .send_keys(" ")\
                .perform()
            time.sleep(0.1)

def risk_checker(text):
    others_check = "False"
    for i in cr_keyword:
        if i in text:
            risk = 'Credit Risk'
            return risk
        

    for i in lr_keyword:
        if i in text:
            lr_check = 'True'
            risk = 'Liquidity Risk'
            return risk

    for i in mr_keyword:
        if i in text:
            mr_check = 'True'
            risk = 'Market Risk'
            return risk

    for i in tr_keyword:
        if i in text:
            tr_check = 'True'
            risk = 'Technology Risk'
            return risk

    for i in or_keyword:
        if i in text:
            or_check = 'True'
            risk = 'Operation Risk'
            return risk

    risk = 'Unidentified'    
    return risk

def word_risk_checker(text):
    others_check = "False"

    if text in cr_keyword:
        risk = 'Credit Risk'
        return risk

    if text in lr_keyword:
        lr_check = 'True'
        risk = 'Liquidity Risk'
        return risk

    if text in mr_keyword:
        mr_check = 'True'
        risk = 'Market Risk'
        return risk

    if text in tr_keyword:
        tr_check = 'True'
        risk = 'Technology Risk'
        return risk

    if text in or_keyword:
        or_check = 'True'
        risk = 'Operation Risk'
        return risk

    risk = 'Unidentified'    
    return risk

def words_segmentation(keywords_set, foul_keywords, skip_keywords, context, comment_uid, comment_datetime):
    #use the customized word list for the nlp
    segmenter = Segmenter(allow= keywords_set,max_word_length=8)
    comment_foul = "False"
    idx = 0
    context = context.split(" ")
    
    for block_context in context:
        #segment the sentence
        segmented_list = pycantonese.segment(block_context, cls=segmenter)
        segmented_list = pycantonese.pos_tag(segmented_list)

        for word, pos in segmented_list:
            word = word.title()
            if word in skip_keywords:
                continue

            if pos == 'ADJ' or pos == 'NOUN' or pos == 'VERB':
                if word in foul_keywords:
                    f_word = "True"
                    comment_foul = "True"
                else:
                    f_word = "False"
                #check chi or eng
                lang, score = langid.classify(word)
                
                word_uid = comment_uid+ '-' + str(idx).zfill(8)
                idx = idx + 1 
                
                '''
                #generate sentiment score
                snow_stm = SnowNLP(word)
                sentiment_score = snow_stm.sentiments
                
                #Sentiment Mood
                sentiment = "Neutral"
                if sentiment_score >= 0.6:
                    sentiment = "Positive"
                if sentiment_score <= 0.4:
                    sentiment = "Negative"
                '''
                word_risk = word_risk_checker(word)
                #t_word = translate_to_en(word)
                #t_sentiment, t_sentiment_score = word_sentiment_check(t_word)
                
                df_words.loc[len(df_words)]= [word_uid,
                                              comment_uid,
                                              comment_datetime,   
                                              lang,
                                              pos,
                                              word.title(),
                                              f_word.title(),
                                              word_risk
                                              #sentiment,
                                              #sentiment_score,
                                              #str(t_word).title(),
                                              #t_sentiment,
                                              #t_sentiment_score
                                             ]
    return comment_foul


def translate_to_en(text):
    try:
        #translate to english
        translated = GoogleTranslator(source='zh-TW', target='en').translate(text)
    except:
        translated = text
    return translated

###############################
#Not used due to not effactive 

def sentiment_check(text):
    try:
        doc = nlp(text)
        polarity = doc._.blob.polarity            
        subjectivity = doc._.blob.subjectivity

        if polarity >= 0.15:
            sentiment = 'Positive'
        elif polarity <= -0.15:
            sentiment = 'Negative'
        else:
            sentiment = 'Neutral'

        if subjectivity >= 0.5:
            perception = 'Subjective'
        else:
            perception = 'Objective'
    except:
        sentiment = 'Neutral'
        polarity = 0
        perception = 'Objective'
        subjectivity = 0
        
    return sentiment, polarity, perception, subjectivity

##############################
#Not used...
def word_sentiment_check(text):
    try:
        doc = nlp(text)
        polarity = doc._.blob.polarity            
        if polarity >= 0.15:
            sentiment = 'Positive'
        elif polarity <= -0.15:
            sentiment = 'Negative'
        else:
            sentiment = 'Neutral'
    except:
        sentiment = 'Neutral'
        polarity = 0
 
    return sentiment, polarity
###############################
                                              
def vader_sentiment(text):
    try:
        sent_analyzer = SentimentIntensityAnalyzer()
        result = sent_analyzer.polarity_scores(text)
        score = result['compound']
        sentiment = "neutral"

        if(result['compound']>= 0.05):
            sentiment = "positive"

        elif(result['compound']<= -0.05):
            sentiment = "negative"
    except:
        sentiment = "neutral"
        score = '0'
    return sentiment, score

###################
#OLD VERSION
'''
# NEED TO CHECK IF THE THREAD EXIST OR NOT FIRST
# Return True == need to update
# Return False == skip it
def left_panel_date_check(previous_extract_latest_comment_time, embedded_date):
    checker = False
    current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    current_datetime = datetime.strptime(current_datetime, "%Y-%m-%d %H:%M:%S")
    splited_embedded_date = embedded_date.split(" ")
    
    previous_extract_latest_comment_time = datetime.strptime(previous_extract_latest_comment_time, "%Y-%m-%d %H:%M:%S")
    
    if splited_embedded_date[1] == '分鐘前':
        decrypted_embedded_datetime = current_datetime - relativedelta(hours = int(splited_embedded_date[0]))
        if previous_extract_latest_comment_time < decrypted_embedded_datetime:
            checker = True
            return checker
        else:
            checker = False
            return checker
 
    elif splited_embedded_date[1] == '小時前':
        decrypted_embedded_datetime = current_datetime - relativedelta(hours = int(splited_embedded_date[0]))
        if previous_extract_latest_comment_time < decrypted_embedded_datetime:
            checker = True
            return checker
        else:
            checker = False
            return checker
    else:
        checker = False
        return checker
'''
###################

def left_panel_date_check(previous_extract_latest_comment_time, embedded_date):
    checker = False
    current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    current_datetime = datetime.strptime(current_datetime, "%Y-%m-%d %H:%M:%S")
    splited_embedded_date = embedded_date.split(" ")
    
    previous_extract_latest_comment_time = datetime.strptime(previous_extract_latest_comment_time, "%Y-%m-%d %H:%M:%S")
    
    if splited_embedded_date[1] == '分鐘前':
        decrypted_embedded_datetime = current_datetime - relativedelta(hours = int(splited_embedded_date[0]))
        print('P1')
        print(previous_extract_latest_comment_time, decrypted_embedded_datetime)
        if previous_extract_latest_comment_time < decrypted_embedded_datetime:
            checker = True
            return checker
        else:
            checker = False
            return checker
 
    elif splited_embedded_date[1] == '小時前':
        if int(splited_embedded_date[0]) > 1:
            checker = False
            return checker
        else:
            checker = True
            return checker
    else:
        checker = False
        return checker
    
def check_thread_date_range(soup):
    all_comment = soup.find_all("div",{"class":"_36ZEkSvpdj_igmog0nluzh"})
    floor_zero = all_comment[0]
    floor_zero_datetime = floor_zero.find("span",{"class":"Ahi80YgykKo22njTSCzs_"})
    new_floor_zero_datetime = str(floor_zero_datetime).split("\"")[5]
    new_floor_zero_datetime = new_floor_zero_datetime.replace('年',"-").replace('月', "-").replace('日', "")
    floor_zero_date = new_floor_zero_datetime.split(" ")[0]


    floor_zero_thread_time = datetime.strptime(floor_zero_date, '%Y-%m-%d').date()
    floor_zero_cur_time = datetime.now().strftime("%Y-%m-%d")
    floor_zero_cur_time = datetime.strptime(floor_zero_cur_time, '%Y-%m-%d').date()
    
    date_out_of_range = False
    if floor_zero_thread_time < floor_zero_cur_time - relativedelta(months=12):
        date_out_of_range = True
        print('OUT OF RANGE... break')
    return date_out_of_range

In [3]:
#Open pandas
ma_storage_dsl_path = '/mnt/prototypehkmastorage1/Web-Scraping SuT/LIHKG/Storage/'
try:
    df_thread_log = pd.read_excel(f'{ma_storage_dsl_path}Thread_Log.xlsx')
except:
    df_thread_log = pd.DataFrame(columns=["Platform UID",
                                       "Thread UID (PK)",
                                       "Search Keyword",
                                       "Reference Bank",
                                       "Thread Create Datetime",
                                       "Thread Author",
                                       "Thread Title",
                                       "Thread Theme",
                                       "Comment Floor Number (Latest)",
                                       "Risk Type",
                                       "Thread Latest Comment Time"])
    
try:
    df_fetch_log = pd.read_excel(f'{ma_storage_dsl_path}Fetch_Log.xlsx')
except:
    df_fetch_log = pd.DataFrame(columns=["Fetch UID (PK)",
                                       "Thread UID (FK)",
                                       "Fetch Datetime",
                                       "Thread Latest Comment Time(Thread info)",
                                       "Thread Latest Comment Time(Fetch inside comment)",
                                       "Thread Number of Page",
                                       "Number of Share",
                                       "Number of Reactions",                                    
                                       "Thread Likes",
                                       "Thread Dislike",
                                       "Angry", #
                                       "Heart", #
                                       "HaHa", #
                                       "Go For It", #
                                       "Cry", #
                                       "WOW", #
                                       "Number of Comment"])

#Get Distinct value by last row

try:
    df_distinct_thread_by_latest = df_thread_log.drop_duplicates(subset=[ 'Thread Author',
                                                                          'Thread Title',
                                                                          'Thread Theme'], keep='last')
except:
    pass
    
try:
    df_comments = pd.read_excel(f'{ma_storage_dsl_path}Comment_Log.xlsx')
except:
    df_comments = pd.DataFrame(columns=["Thread UID (FK)",
                                        "Comment UID (PK)",
                                        "Fetch Datetime",
                                        "Comment Floor Number",
                                        "Comment Author",
                                        "Original Embedded Date", #
                                        "Comment Datetime",
                                        "Comment Date",
                                        "Comment Time",
                                        "Comment Likes",
                                        "Comment Dislike",
                                        "Comment Replies",
                                        "Comment Context",
                                        "Comment Context (Translated)",
                                        "Sentiment",
                                        "Sentiment Score",
                                        "Risk Type (Comment Level)",
                                        #"Perception",
                                        #"Perception Score",
                                        "Offensive Comment"])   
    
try:
    df_words = pd.read_excel(f'{ma_storage_dsl_path}Segmented_Words.xlsx')
except:
    df_words = pd.DataFrame(columns=["Word UID (PK)",
                                   "Comment UID (FK)",
                                   "Datetime (Comment)",
                                   "Language",
                                   "Part of Speech Tagging",
                                   "Word",
                                   "Foul Language",
                                   "Risk Type (Word Level)"])
                                   #"Sentiment",
                                   #"Sentiment Score",
                                   #"Word (Translated)",
                                   #"Sentiment (Translated)",
                                   #"Sentiment Score (Translated)"
                                    
######
# Segment Words
#


ma_required_file_dsl_path = '/mnt/prototypehkmastorage1/Web-Scraping SuT/LIHKG/Required_Files/'

#Get Customized Keyword For NLP Checks
df_keywords_NLP = pd.read_excel(f'{ma_required_file_dsl_path}Keywords_NLP.xlsx', index_col=None)
keywords_NLP = df_keywords_NLP["Keyword"].values.tolist()

#Set empty set to store keywords and used by the pycantonese
keywords_NLP_set = set()
for i in keywords_NLP:
    keywords_NLP_set.add(i)
    
#Get Foul Words
df_foul_keywords = pd.read_excel(f'{ma_required_file_dsl_path}Keywords_Foul_Language.xlsx', index_col=None)
foul_keywords = df_foul_keywords["Keyword"].values.tolist()

#Get Skip Words
df_skip_keywords = pd.read_excel(f'{ma_required_file_dsl_path}Keywords_Skip.xlsx', index_col=None)
skip_keywords = df_skip_keywords["Keyword"].values.tolist()

#Set language pack - lang identifer
langid.set_languages(['zh', 'en'])
#
#
###############

#Set Vader
sent_analyzer = SentimentIntensityAnalyzer()

#Set language pack - Spacy
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')




df_risk_type = pd.read_excel(f'{ma_required_file_dsl_path}Keywords_Risk.xlsx')
cr_keyword = df_risk_type["CR"].dropna().values.tolist()
lr_keyword = df_risk_type["LR"].dropna().values.tolist()
mr_keyword = df_risk_type["MR"].dropna().values.tolist()
tr_keyword = df_risk_type["TR"].dropna().values.tolist()
or_keyword = df_risk_type["OR"].dropna().values.tolist()
    
df_keyword = pd.read_excel(f'{ma_required_file_dsl_path}LIHKG_Keyword.xlsx', index_col=None)
keyword_list = df_keyword["Keyword"].values.tolist()
ref_bank_list = df_keyword["Reference Bank"].values.tolist()
df_keyword

,Keyword,Reference Bank
0,渣打,Standard Chartered Hong Kong
1,揸兜,Standard Chartered Hong Kong
2,滙豐,Hongkong and Shanghai Banking Corporation
3,匯豐,Hongkong and Shanghai Banking Corporation
4,hsbc,Hongkong and Shanghai Banking Corporation
5,大貓銀行,Hongkong and Shanghai Banking Corporation
6,行山銀行,Hang Seng Bank Limited
7,恆生,Hang Seng Bank Limited
8,花旗銀行,Citibank (Hong Kong)
9,citi,Citibank (Hong Kong)


In [ ]:
%%time
########
# MAIN #
########

#set driver anti detect
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_experimental_option(
    "excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_argument('lang=zh-CN,zh,zh-TW,en-US,en')
chrome_options.add_argument(
    'user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36')
chrome_options.add_argument("disable-blink-features=AutomationControlled")
#proxy
'''
chrome_options.add_argument("proxy-server=socks5://127.0.0.1:1080")
preferences = {
    "webrtc.ip_handling_policy": "disable_non_proxied_udp",
    "webrtc.multiple_routes_enabled": False,
    "webrtc.nonproxied_udp_enabled": False
}
'''

chrome_drive_dsl_path = '/mnt/prototypehkmastorage1/Web-Scraping SuT/LIHKG/Required_Files/chromedriver'

#starts of the loop
for keyword_row_num, keyword in enumerate(keyword_list):
    print("Keyword Check")
    
    #start Chrome
    driver = webdriver.Chrome(chrome_drive_dsl_path, options=chrome_options)  
    
    print("Loop",keyword_row_num,keyword)
    #Start of fetch
    ref_bank = ref_bank_list[keyword_row_num]
    fetch_datetime = datetime.now().strftime("%Y%m%d-%H%M%S%f")
    #keyword_uid = "LI-"+str(fetch_datetime)+"-"+ str(keyword_row_num).zfill(8)
    platform_uid = "LI"
    fetch_uid = platform_uid+"-"+ str(fetch_datetime)

    #get link
    driver.get(f"https://lihkg.com/search?q={keyword}&sort=desc_create_time&type=thread")
    time.sleep(4)
    
    #get html info
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    #click left panel for scrolling
    driver.find_element(By.XPATH, '//*[@id="leftPanel"]/div[1]/ul/li[2]/a').click()

    #Call the function and scroll to the end (leftpanel)
    scroll_left_panel(driver)
    time.sleep(2)
    
    #get latest html info
    html = driver.page_source
    soup = BeautifulSoup(html)
    time.sleep(2)
    
    print("Start - loop thread")
    #get thread html info
    threads = soup.find("div",{"class":"qoAmEqNpZRLf2KVKZ8DsC"})
    for thread_idx, i in enumerate(threads):
        '''
        ## CUSTOM STOP
        if thread_idx == 3:
            break
        '''
        
        #click thread
        click_thread(driver, thread_idx)
        #give buffer incase the webpage use too much resource.
        time.sleep(1)
        
        #get thread info
        try:
            try:
                thread_author = i.find("span", {"class":"CxY4XDSSItTeLVg0cKCN0 A0jheqYUBHNW93KykXKEH"}).text
            except:
                thread_author = i.find("span", {"class":"CxY4XDSSItTeLVg0cKCN0 jj_3ZDzjtPixL1b2KTcpS"}).text

            thread_lastest_comment_time = i.find("span", {"class":"_37XwjAqVHtjzqzEtybpHrU"}).text
            thread_lastest_comment_time.replace('年',"-").replace('月', "-").replace('日', "")
            
            ###
            #check day or hr or month function & break it when the time is not in range
            #
            #
            ###

            thread_title = i.find("span", {"class":"_20jopXBFHNQ9FUbcGHLcHH"}).text
            thread_page = i.find("div", {"class":"_26oEXjfUS_iHzbxYcZE6bD"}).text.split(" ")[0]
            thread_theme = i.find_all("a")[1].text
            thread_total_like = driver.find_elements(By.CLASS_NAME, '_8_NT40G-QNQzcSSTrRXAD')[0].get_attribute('data-score')
            thread_total_dislike = driver.find_elements(By.CLASS_NAME, '_8_NT40G-QNQzcSSTrRXAD')[1].get_attribute('data-score')
            risk_type = risk_checker(str(thread_title).lower())
            total_reaction = int(thread_total_like) + int(thread_total_dislike)
            
            angry = 'Null'
            heart = 'Null'
            haha ='Null'
            goforit = 'Null'
            cry = 'Null'
            wow = 'Null'
            org_emb_date = 'Null'
            total_share = 'Null'
        except:
            continue
        
        #Locate the thread in excel... if not exsit then keep going.. else exist and no update will skip it...
        update_checker = True
        try:
            thread_exist_loc = df_distinct_thread_by_latest.loc[(df_distinct_thread_by_latest['Thread Author'] == thread_author) 
                                                                & (df_distinct_thread_by_latest['Thread Title'] == thread_title)
                                                                & (df_distinct_thread_by_latest['Thread Theme'] == thread_theme)]

            floor_number_pointer = thread_exist_loc["Comment Floor Number (Latest)"].values[0]
            previous_extract_latest_comment_time_pointer = thread_exist_loc["Thread Latest Comment Time"].values[0]
            print(f'Found... {floor_number_pointer}, {previous_extract_latest_comment_time_pointer}')

            # Compare the time
            # 1 Return True == need to update
            # 2 Return False == skip it
            update_checker = left_panel_date_check(previous_extract_latest_comment_time_pointer, thread_lastest_comment_time)
            if update_checker == True:
                print(update_checker, 'need updates')
            
            # if == to false the program stops after checking the post is it within the date range below...
            if update_checker == False:
                print(update_checker, 'no updates... check for the post date...')
                #continue


        except:
            thread_exist = False
            floor_number_pointer = '#0'
            print('Not found... Set floor pointer to 0')
    
            
            
        #current time
        fetch_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        #get the latest page info
        html = driver.page_source
        soup = BeautifulSoup(html)
        
        #check thread in range or not
        date_out_of_range = check_thread_date_range(soup)
        if date_out_of_range == True:
            print('Alert - Thread created date out of customized time range... Skip it...')
            break
        
        # if no need update... skip after checking the date
        if update_checker == False:
            print(update_checker, 'Didnt exceed the range... continue the loop ')
            continue
        
        #check the keyword is in the title or not
        if str(keyword).lower() not in str(thread_title).lower():
            print('Keyword not included in title... Skip it...')
            continue        

        print("Start - Loop to end of the post")
        #loop to the end of pages
        for click in range(int(thread_page)+3):
            print(f"{int(thread_page)+3} - Looping Check")
            try:
                find_f5 = driver.find_element(By.CLASS_NAME, '_1PhR8JHkMcET5QT2PtCA3T')
                actions = ActionChains(driver)
                actions.move_to_element(find_f5).perform()
            except:
                try:
                    find_f5 = driver.find_element(By.XPATH, f'//*[@id="page-{str(click+1)}"]/div[3]/a').click()
                except:
                    for j in range(20):
                        ActionChains(driver).send_keys(" ").perform()
            time.sleep(0.3)
        print("End - loop to end of post")

        time.sleep(1)                
        #get the latest page info
        html = driver.page_source
        soup = BeautifulSoup(html)

        print("Start - Process Comment")
        #start of fetching comments data
        all_comment = soup.find_all("div",{"class":"_36ZEkSvpdj_igmog0nluzh"})
        for comment_idx , comment in enumerate(all_comment):
            print("Loop - Process Comment")
            comment_floor_number = comment.find_all("span")[0].text
            comment_author = comment.find("span",{"class":"ZZtOrmcIRcvdpnW09DzFk"}).text

            #incase the comment is folded due to not enough info
            try:
                comment_datetime = comment.find("span",{"class":"Ahi80YgykKo22njTSCzs_"})
                new_comment_datetime = str(comment_datetime).split("\"")[5]
                new_comment_datetime = new_comment_datetime.replace('年',"-").replace('月', "-").replace('日', "")
                comment_date = new_comment_datetime.split(" ")[0]
                comment_time = new_comment_datetime.split(" ")[1]

                comment_like = comment.find_all("label")[1].text
                comment_dislike = comment.find_all("label")[3].text
            except:
                comment_datetime = ""
                comment_date = ''
                comment_time = ''
                comment_like = 0
                comment_dislike = 0

            try:
                comment_replies = comment.find_all("label")[5].text
            except:
                comment_replies = 0

            try:
                comment_context = comment.find("div",{"class":"_2cNsJna0_hV8tdMj3X6_gJ"}).text
            except:
                comment_context = ''

            
            #find earliest time
            if comment_idx == 0:
                thread_post_datetime = new_comment_datetime
                
                #thread UID
                thread_uid = thread_author + "-" + thread_theme + "-" + thread_title + "-" + thread_post_datetime

            
            comment_uid = thread_uid + "-" + str(comment_floor_number.replace("#", "")).zfill(8)

            #get the lastest comment time
            latest_comment_time = new_comment_datetime
                
            #get lastest floor number in this fetch
            lastest_floor_number = comment_floor_number
            

            
            #current time
            fetch_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            
            #compare the latest floor number
            if int(floor_number_pointer.replace("#", "")) < int(comment_floor_number.replace("#", "")):
                print(floor_number_pointer, comment_floor_number)
                comment_updates = True
                ### do the word segmentation and get comment contain foul or not
                comment_foul = words_segmentation(keywords_NLP_set,
                                                  foul_keywords,
                                                  skip_keywords,
                                                  comment_context,
                                                  comment_uid,
                                                  new_comment_datetime)
                
                translated_context = translate_to_en(comment_context)
                
                #####
                #Scrapy Sentiment check
                #sentiment, sentiment_score, perception, perception_score = sentiment_check(translated_context)
                #####
                
                #Vader Sentiment Check
                sentiment, sentiment_score = vader_sentiment(translated_context)
                #Risk Type
                comment_risk_type = risk_checker(comment_context)
                
                #total comment (by this fetch)
                num_of_comment = int(lastest_floor_number.replace("#", ''))

                
                df_comments.loc[len(df_comments)]= [thread_uid,
                                                     comment_uid,
                                                     fetch_datetime,
                                                     comment_floor_number,
                                                     comment_author,
                                                     org_emb_date,
                                                     new_comment_datetime,
                                                     comment_date,
                                                     comment_time,
                                                     int(comment_like),
                                                     int(comment_dislike), 
                                                     int(comment_replies), 
                                                     comment_context,
                                                     translated_context,
                                                     sentiment, 
                                                     sentiment_score, 
                                                     comment_risk_type,
                                                     #perception, 
                                                     #perception_score,                                                    
                                                     comment_foul]
                
            else:
                comment_updates = False
        print("After process all comment in a thread ")
        
        
        if comment_updates == True:
            #save thread into to dataframe
            df_thread_log.loc[len(df_thread_log)]= [platform_uid,
                                              thread_uid,
                                              keyword,
                                              ref_bank,                                         
                                              thread_post_datetime,
                                              thread_author, 
                                              thread_title,
                                              thread_theme,
                                              lastest_floor_number,
                                              risk_type,
                                              latest_comment_time]



            df_fetch_log.loc[len(df_fetch_log)]= [fetch_uid,
                                              thread_uid,
                                              fetch_datetime,                                          
                                              thread_lastest_comment_time,
                                              latest_comment_time,
                                              int(thread_page),
                                              total_share,
                                              total_reaction,
                                              int(thread_total_like), 
                                              int(thread_total_dislike),
                                              angry,
                                              heart,
                                              haha,
                                              goforit,
                                              cry,
                                              wow,
                                              num_of_comment]
            
            print("End of this thread Check")
    
    '''
        if thread_idx == 4:
            break
    
    if keyword_row_num == 1:
        break
    '''
    
    driver.quit()
    time.sleep(1)

Keyword Check


/home/hkmard/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


Loop 0 渣打
Start - loop thread
Not found... Set floor pointer to 0
Start - Loop to end of the post
5 - Looping Check
5 - Looping Check
5 - Looping Check
5 - Looping Check
5 - Looping Check
End - loop to end of post
Start - Process Comment
Loop - Process Comment
#0 #1
Loop - Process Comment
#0 #2
Loop - Process Comment
#0 #3
Loop - Process Comment
#0 #4
Loop - Process Comment
#0 #5
Loop - Process Comment
#0 #6
Loop - Process Comment
#0 #7
Loop - Process Comment
#0 #8
Loop - Process Comment
#0 #9
Loop - Process Comment
#0 #10
Loop - Process Comment
#0 #11
Loop - Process Comment
#0 #12
Loop - Process Comment
#0 #13
Loop - Process Comment
#0 #14
Loop - Process Comment
#0 #15
Loop - Process Comment
#0 #16
Loop - Process Comment
#0 #17
Loop - Process Comment
#0 #18
Loop - Process Comment
#0 #19
Loop - Process Comment
#0 #20
Loop - Process Comment
#0 #21
Loop - Process Comment
#0 #22
Loop - Process Comment
#0 #23
Loop - Process Comment
#0 #24
Loop - Process Comment
#0 #25
Loop - Process Commen

In [ ]:
df_thread_log

In [ ]:
df_fetch_log

In [ ]:
df_comments

In [ ]:
df_words

In [ ]:
#ma_output_storage_path = '/mnt/prototypehkmastorage1/Web-Scraping SuT/LIHKG/Storage/'

#output data
#drop duplicated thread
df_thread_log.drop_duplicates(keep = 'last')
df_thread_log.to_excel(f'{ma_storage_dsl_path}Thread_Log.xlsx',encoding= 'utf-8-sig',index = None) 

df_fetch_log.to_excel(f'{ma_storage_dsl_path}Fetch_Log.xlsx',encoding= 'utf-8-sig',index = None) 
df_comments.to_excel(f'{ma_storage_dsl_path}Comment_Log.xlsx',encoding= 'utf-8-sig',index = None) 
df_words.to_excel(f'{ma_storage_dsl_path}Segmented_Words.xlsx',encoding= 'utf-8-sig',index = None) 

In [ ]:
#read and save it without null, need to find another way be smart
df_comments = pd.read_excel(f'{ma_storage_dsl_path}Comment_Log.xlsx')
#drop null value
df_comments = df_comments.dropna()
df_comments.to_excel(f'{ma_storage_dsl_path}Comment_Log.xlsx',encoding= 'utf-8-sig',index = None) 
